In [1]:
from segment_anything import SamPredictor, sam_model_registry
import numpy as np
import torch
import cv2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_SAM = sam_model_registry["vit_h"](checkpoint="/home/jacobo15defrutos/AVS9/sam_vit_h_4b8939.pth")
model_SAM.to(device)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [3]:
import re

def find_number_in_string(input_string):
    # Regular expression to find a number in a string
    pattern = r'\d+'

    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    # Check if a match is found
    if match:
        # Extract and return the matched number
        return int(match.group())

    # Return None if no number is found
    return None

In [5]:
# Preprocess the images
from collections import defaultdict
import torch
from segment_anything.utils.transforms import ResizeLongestSide
import os

data_folder_train= '/home/jacobo15defrutos/AVS9/Data/Data_seg_SAM/train'
data_folder_val= '/home/jacobo15defrutos/AVS9/Data/Data_seg_SAM/val'
transformed__train_data_imgs= defaultdict(dict)
transformed__train_data_masks= defaultdict(dict)
list_numbers=[]
for folder in os.listdir(data_folder_train):
    print("-"*20)
    print("Preprocessing: ",folder)
    for file in os.listdir(f'{data_folder_train}/{folder}'):
      print(file)
      if 'png'  in file:
        path = f'{data_folder_train}/{folder}/{file}'
        print(path)
        image = cv2.imread(path)
        image = cv2.resize(image, (800, 800))
        if 'Imag'  in folder:
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          #print(image.shape)
          transform = ResizeLongestSide(model_SAM.image_encoder.img_size)
          #print("mierda: ", model_SAM.model.image_encoder.img_size)
          input_image = transform.apply_image(image)
          input_image_torch = torch.as_tensor(input_image, device=device,dtype=torch.float32)
          #print(input_image_torch.shape)
          #expected_size = (1, 3, model_SAM.model.image_encoder.img_size, model_SAM.model.image_encoder.img_size)
          #transformed_image = input_image_torch.view(expected_size)
          transformed_image = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
          #print(transformed_image.shape)
          input_image = model_SAM.image_encoder(transformed_image)#[1,256,64,64]
          original_image_size = image.shape[:2]
          #print("Original: ", original_image_size)
          input_size = tuple(transformed_image.shape[-2:])
          number= find_number_in_string(file)
          #print("nUmber: ",number)
          list_numbers.append(number)
          transformed__train_data_imgs[number]['image'] = input_image
          transformed__train_data_imgs[number]['input_size'] = input_size
          transformed__train_data_imgs[number]['original_image_size'] = original_image_size
        elif 'Labels'in folder:
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image= image[:,:,1]
          #print(image.shape)
          transform = ResizeLongestSide(model_SAM.image_encoder.img_size)
          #print("mierda: ", model_SAM.model.image_encoder.img_size)
          input_image = transform.apply_image(image)
          input_image_torch = torch.as_tensor(input_image, device=device,dtype=torch.uint8)
          input_image_torch= input_image_torch.unsqueeze(2)
          #print(input_image_torch.shape)
          #expected_size = (1, 3, model_SAM.model.image_encoder.img_size, model_SAM.model.image_encoder.img_size)
          #transformed_image = input_image_torch.view(expected_size)
          transformed_image = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]
          #print(transformed_image.shape)
          #input_image = model_SAM.model.image_encoder(transformed_image)
          original_image_size = image.shape[:2]
          #print("Original: ", original_image_size)
          input_size = tuple(transformed_image.shape[-2:])
          number= find_number_in_string(file)
          #print("nUmber: ",number)
          transformed__train_data_masks[number]['image'] = transformed_image
          #transformed__train_data_masks[number]['input_size'] = input_size
          #transformed__train_data_masks[number]['original_image_size'] = original_image_size

--------------------
Preprocessing:  Imag
Captura_de_pantalla_(35).png
/home/jacobo15defrutos/AVS9/Data/Data_seg_SAM/train/Imag/Captura_de_pantalla_(35).png
Captura_de_pantalla_(54).png
/home/jacobo15defrutos/AVS9/Data/Data_seg_SAM/train/Imag/Captura_de_pantalla_(54).png


: 